In [1]:
import pickle, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
resDir = '/egr/research-aidd/menghan1/AnchorDrug/resultBaseLine/advbim_tuning'
files = [f for f in os.listdir(resDir) if f.endswith('balancesample.csv')]
querys = list(set([f.split('_')[5] for f in files]))
eps = [float(q.split('-')[3]) for q in querys]
querys = [x for _, x in sorted(zip(eps, querys))]
print(querys)

['AdversarialBIM-0.7-0.8-0.0003',
 'AdversarialBIM-0.7-0.8-0.0005',
 'AdversarialBIM-0.7-0.8-0.0007',
 'AdversarialBIM-0.7-0.8-0.0009',
 'AdversarialBIM-0.7-0.8-0.0011',
 'AdversarialBIM-0.7-0.8-0.0013',
 'AdversarialBIM-0.7-0.8-0.0015']

In [4]:
celllist = ['A549', 'MCF7', 'PC3']
for c in celllist:
    print(c)
    filesc = [f for f in files if c in f]
    # [print(fq) for fq in filesc]
    f1_all, precision_all, recall_all = [],[],[]
    df_c = pd.DataFrame()
    for q in querys:
        fqc = [f for f in filesc if q in f][0]
        print(q)
        df = pd.read_csv(f'{resDir}/{fqc}', index_col=0)
        # print(df)
        # print(df.mean(1))
        df_c[q] = df.mean(1)
        # print(df.T.to_numpy().mean(0))
        # print(df.T.to_numpy().std(0))
    print(df_c.T)
    df_c.T.to_csv(f'{resDir}/all_summary_{c}.csv')

A549
AdversarialBIM-0.7-0.8-0.0003
AdversarialBIM-0.7-0.8-0.0005
AdversarialBIM-0.7-0.8-0.0007
AdversarialBIM-0.7-0.8-0.0009
AdversarialBIM-0.7-0.8-0.0011
AdversarialBIM-0.7-0.8-0.0013
AdversarialBIM-0.7-0.8-0.0015
                                    acc        f1  precision label 0  \
AdversarialBIM-0.7-0.8-0.0003  0.701892  0.546805           0.403893   
AdversarialBIM-0.7-0.8-0.0005  0.696985  0.547143           0.411077   
AdversarialBIM-0.7-0.8-0.0007  0.706231  0.547384           0.413454   
AdversarialBIM-0.7-0.8-0.0009  0.695045  0.545390           0.411731   
AdversarialBIM-0.7-0.8-0.0011  0.706341  0.549384           0.409577   
AdversarialBIM-0.7-0.8-0.0013  0.703569  0.547376           0.402580   
AdversarialBIM-0.7-0.8-0.0015  0.703056  0.548367           0.405854   

                               precision label 1  precision label 2  \
AdversarialBIM-0.7-0.8-0.0003           0.826394           0.377485   
AdversarialBIM-0.7-0.8-0.0005           0.828797           0.36332